# Similitud en ontologías

Una ontología es una representación gráfica del conocimiento que cuenta con conceptos, instancias y relaciones. Los términos (instancias) de una ontología se relacionan mediante relaciones léxicas, principalmente de hiperonimia e hiponimia.

Aquí revisamos la ontología WordNet y cómo se puede usar ésta para calcular la similitud entre términos.

In [1]:
import nltk
from nltk.corpus import wordnet

### WordNet

WorNet es una ontología de uso general que contiene términos en inglés, se puede consultar en <a>https://wordnet.princeton.edu/</a>. En este caso, usamos la biblioteca de NLTK para descargar la ontología.

In [2]:
#Descargamos WordNet con NLTK
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

WordNet en NLTK tiene varias funciones interesantes que pueden consultarse en <a>https://www.nltk.org/howto/wordnet.html</a>. Algunas de las que usaremos son:

* synset: se aplica a una cadena (str) para obtener la instancia dentro de la ontología. Las cadenas son del tipo 'cat.n.01' donde 'cat' es la palabra a buscar 'n' es una etiqueta POS (indica que 'cat' es un sustantivo 'n' o un verbo 'v' o adjetivo 'a', etc.); finalmente, se enumeran los homónimos, si dos palabras suenan igual pero refieren a entidades distintias se usa un indicador distinto ('01', '02', etc.)
* name: Refiere al nombre dado al término de búsqueda como entrada del diccionario.
* definition: Obtiene la definición de diccionario de la palabra.
* hypernyms: Obtiene los hiperónimos de la palabra; es decir, los términos arriba de la palabra bajo la relación is_a.
* hyponyms: Obtiene los hipónimos de la palabra; es decir, los términos abajo de la palabra bajo la relación is_a.
* antonyms: Obtiene los antónimos de un lema.

In [3]:
#Buscamos la palabra 'cat' sustantivo 'n'
cat = wordnet.synset('cat.n.01')
#Imprimimos el nombre y la definición
print('{}. Definición: {}\n'.format(cat.name(), cat.definition()))
#Imprimimos los hiperónimos y los hipónimos
print('Hiperónimos: ',[hyper.name() for hyper in cat.hypernyms()])
print('Hipónimos: ',[hypo.name() for hypo in cat.hyponyms()])

#Para consultar los antónimos, necesitamos
#aplicar la función antonyms() a los lemmas
good = wordnet.synset('good.a.01')
print('Antónimos de {}: {}'.format(good.name(), good.lemmas()[0].antonyms()))

cat.n.01. Definición: feline mammal usually having thick soft fur and no ability to roar: domestic cats; wildcats

Hiperónimos:  ['feline.n.01']
Hipónimos:  ['domestic_cat.n.01', 'wildcat.n.03']
Antónimos de good.a.01: [Lemma('bad.a.01.bad')]


## Cálculo de la similitud

Para calcular la similitud definiremos una función que obtiene los términos que forman el camino desde la raíz de la ontología hasta el término actual de la consulta.

In [4]:
def get_path(w_in, root='entity.n.01'):
  """
  Obtiene el camino que va de la raíz de la ontología (término más general)
  hasta el término búscado.

  Arguments
  ---------
  w_in : str
    Término a consultar
  root : str
    Término raíz, puede variar según la POS

  Returns
  -------
  path : list
    Lista de hiperónimos que van desde la raíz hasta el término
  """
  #Obtiene en synset
  w =  wordnet.synset(w_in)
  #Guarda el camino
  path = []
  #El término raíz es entity.n.01 en sustantivos
  while w.name() != root:
    #Genera los hiperónimos
    hyp_w = w.hypernyms()[0].name()
    w = w.hypernyms()[0]
    #Guarda el camino de hiperónimos
    path.append(hyp_w)

  return path[::-1] + [w_in]

print(get_path('domestic_cat.n.01'))

['entity.n.01', 'physical_entity.n.01', 'object.n.01', 'whole.n.02', 'living_thing.n.01', 'organism.n.01', 'animal.n.01', 'chordate.n.01', 'vertebrate.n.01', 'mammal.n.01', 'placental.n.01', 'carnivore.n.01', 'feline.n.01', 'cat.n.01', 'domestic_cat.n.01']


Podemos tratar de visualizar como se ven los caminos entre dos términos, ya que la similitud se calculará en base en esto.

In [5]:
def onto_path(w1, w2):
  """
  Visualiza el camino entre dos térmninos, viendo los términos en común y los disjuntos.

  Arguments
  ---------
  w1, w2 : str
    Términos de los que se desea visualizar los caminos.
  """
  #Obtiene los caminos de los synsets
  path1, path2 = get_path(w1), get_path(w2)
  #El camino más corto
  l = min(len(path1), len(path2))
  
  for i in range(l):
    #Imprime los nodos en común
    if path1[i] == path2[i]:
      print('\t{}'.format(path1[i]))
    #Imprime los nodos diferentes
    else:
      print('{} \t\t {}'.format(path1[i], path2[i]))

  #Si un camino es más largo imprime los faltantes
  if len(path1) > len(path2):
    for i in range(l, len(path1)):
      print(path1[i])
  elif len(path1) < len(path2):
    for i in range(l, len(path2)):
      print('\t \t\t {}'.format(path2[i]))
  else:
    pass

#Visualizamos la ontología para cat y dog
onto_path('cat.n.01', 'dog.n.01')

	entity.n.01
	physical_entity.n.01
	object.n.01
	whole.n.02
	living_thing.n.01
	organism.n.01
	animal.n.01
	chordate.n.01
	vertebrate.n.01
	mammal.n.01
	placental.n.01
	carnivore.n.01
feline.n.01 		 canine.n.02
cat.n.01 		 dog.n.01


In [6]:
#Visualizamos la ontología para car y bus
onto_path('car.n.01', 'bus.n.01')

	entity.n.01
	physical_entity.n.01
	object.n.01
	whole.n.02
	artifact.n.01
	instrumentality.n.03
container.n.01 		 conveyance.n.03
wheeled_vehicle.n.01 		 public_transport.n.01
self-propelled_vehicle.n.01 		 bus.n.01
motor_vehicle.n.01
car.n.01


Para calcular la similitud entre dos términos de la ontología, $w_1$ y $w_2$,  utilizamos la función:

$$sim(w_1, w_2) = 2\frac{common(w_1,w_2)}{depth(w1) + depth(w2)}$$

Donde $common(w_1, w_2)$ es el número de nodos/hiperónimos comúnes entre los términos, esto es: $$common(w_1, w_2) = |\{w : w_1 \text{ is_a } w, w_2 \text{ is_a } w\}|$$

Y $depth(w)$ es la profundidad del término en la ontología, es decir: $$depth(w) = |path(w)|$$

In [7]:
def similarity(w1, w2):
  """
  Cálculo de la similitud entre dos términos a partir de la ontología.

  Arguments
  ---------
  w1, w2 : str
    Términos de los que se obtendrá la similitud

  Returns
  -------
  similarity : float
    Medida que indica la similitud entre ambos términos.
  """
  #Obtiene los caminos
  path1, path2 = get_path(w1), get_path(w2)
  #Calcula la profundidad de cada término
  depth_s1, depth_s2 = len(path1), len(path2)
  #Calcula los nodos en común
  common = len(set(path1).intersection(set(path2)))
  #Computa la similitud
  sim = 2*common/(depth_s1 + depth_s2)

  return sim

In [8]:
#similitud para bus y car
print(similarity('bus.n.01', 'car.n.01'))
#Similitud para cat y dog
print(similarity('cat.n.01', 'dog.n.01'))

0.6
0.8571428571428571
